In [1]:
import pandas as pd
import ast

In [3]:
steam_games = pd.read_csv(r"CSV\steam_games.csv")
user_items =pd.read_csv(r"CSV\user_items.csv")


In [4]:
# Combinar DataFrames utilizando las variables en español
tiempo_juego_genero = pd.merge(steam_games[['id', 'genres', 'release_year']],
                               user_items[['item_id', 'playtime_forever']],
                               left_on='id', right_on='item_id', how='inner')

# Eliminar las columnas duplicadas 'item_id' e 'id'
tiempo_juego_genero = tiempo_juego_genero.drop(columns=['item_id'], axis=1)

In [5]:
tiempo_juego_genero.head()

,id,genres,release_year,playtime_forever
0,282010.0,"['Racing', 'Action', 'Indie']",1997.0,5
1,282010.0,"['Racing', 'Action', 'Indie']",1997.0,0
2,282010.0,"['Racing', 'Action', 'Indie']",1997.0,0
3,282010.0,"['Racing', 'Action', 'Indie']",1997.0,0
4,282010.0,"['Racing', 'Action', 'Indie']",1997.0,13


In [6]:
#Agrupamos por playtime_forever por id
tiempo_juego_genero = tiempo_juego_genero.groupby(['genres', 'release_year', 'id'])['playtime_forever'].sum().reset_index()


In [7]:
tiempo_juego_genero.head()

,genres,release_year,id,playtime_forever
0,"['Action', 'Adventure', 'Casual', 'Indie']",2016.0,499460.0,13
1,"['Action', 'Adventure', 'Casual', 'Indie']",2016.0,514890.0,2
2,"['Action', 'Adventure', 'Casual', 'Strategy']",2015.0,355020.0,0
3,"['Action', 'Adventure', 'Casual']",1999.0,20.0,960524
4,"['Action', 'Adventure', 'Casual']",2009.0,319170.0,5581


In [8]:
#Convertimos la columna genres a tipo lista
tiempo_juego_genero['genres'] = tiempo_juego_genero['genres'].apply(ast.literal_eval)
#Expandimos las listas en columnas
tiempo_juego_genero = tiempo_juego_genero.explode('genres').reset_index(drop=True)

In [9]:
tiempo_juego_genero.head()

,genres,release_year,id,playtime_forever
0,Action,2016.0,499460.0,13
1,Adventure,2016.0,499460.0,13
2,Casual,2016.0,499460.0,13
3,Indie,2016.0,499460.0,13
4,Action,2016.0,514890.0,2


In [ ]:
#Agrupamos por genero y año de lanzamiento
tiempo_juego_genero = tiempo_juego_genero.groupby(['release_year', 'genres'])['playtime_forever'].sum().reset_index()

In [10]:
tiempo_juego_genero.head()

,genres,release_year,id,playtime_forever
0,Action,2016.0,499460.0,13
1,Adventure,2016.0,499460.0,13
2,Casual,2016.0,499460.0,13
3,Indie,2016.0,499460.0,13
4,Action,2016.0,514890.0,2


In [14]:
# Guardar el DataFrame en un archivo CSV
tiempo_juego_genero.to_csv('E:\HENRY\PI MLOps\PI MLOps - STEAM\Api\PlayTimeGenre.csv', index=False)

In [15]:
def PlayTimeGenre(genero):
    """
    Función que toma como entrada un género y devuelve el año con la máxima cantidad de horas jugadas para ese género.

    Parámetros:
    - genero (str): Género para el cual se busca el año con más horas jugadas.

    Ejemplo de retorno:
    - Para un género existente: 2013
    - Para un género no encontrado: "No se encontraron datos para el género 'Género X'"
    """
    # Filtra el DataFrame para el género especificado

    df_genero = tiempo_juego_genero[tiempo_juego_genero['genres'] == genero]

    # Verifica si el género está presente en el DataFrame
    if not df_genero.empty:
        # Encuentra el año con la máxima cantidad de horas jugadas
        anio_mas_jugado = df_genero.loc[df_genero['playtime_forever'].idxmax()]['release_year']
        return f'Año de lanzamiento con más horas jugadas para el género {genero} : {int(anio_mas_jugado)}'
    else:
        return f"No se encontraron datos para el género '{genero}'"